In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
#依赖的包
from lib.dataloader import normal_and_generate_dataset_time, get_mask, get_adjacent, get_grid_node_map_maxtrix
#攻击函数import
from lib.utils_new import mask_loss, compute_loss, predict_and_evaluate, attack, random_attack, fgsm_attack, min_attack
from lib.utils_new import saliency, saliency_loss, attack_js, attack_js,attack_saliency,rand_attack
from model.GSNet import GSNet
from lib.early_stop import EarlyStopping
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import json
import configparser
import pickle as pkl
from time import time
from datetime import datetime
import shutil
import argparse
import random
import math

import sys
import os


/home/wyb/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wyb/miniconda3/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [5]:
#完整运行时会生成txt日志记录#
train_log_filename = "test_log.txt"
train_log_filepath = os.path.join("./", train_log_filename)
now = datetime.now()
date_time = now.strftime("|%Y-%m-%d, %H:%M:%S| ")

#以命令方式运行时会自动设置工作空间
#file会报错，应该家’‘
curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

In [6]:
#配置文件载入，初始参数记录

config_filename = "config/nyc/GSNet_NYC_Config.json"
with open(config_filename, 'r') as f:
    config = json.loads(f.read())
# 写入测试超参数
with open(train_log_filepath, "a") as f:
    f.write('测试文件运行时间：'+date_time)
    f.write('\r\n')
    f.write('超参数设置：')
    f.write('\r\n')
    f.write(json.dumps(config, sort_keys=True, indent=4))
    f.write('\r\n')
f.close
#GPU设置
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

#文件读取
#命令行参数实质上就是赋初始值的作用，可以删除直接赋值就好
north_south_map = config['north_south_map']
west_east_map = config['west_east_map']

all_data_filename = config['all_data_filename']
mask_filename = config['mask_filename']

road_adj_filename = config['road_adj_filename']
risk_adj_filename = config['risk_adj_filename']
poi_adj_filename = config['poi_adj_filename']
grid_node_filename = config['grid_node_filename']
grid_node_map = get_grid_node_map_maxtrix(grid_node_filename)
num_of_vertices = grid_node_map.shape[1]


patience = config['patience']#用处：
delta = config['delta']

if config['seed'] is not None:
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


train_rate = config['train_rate']
valid_rate = config['valid_rate']

recent_prior = config['recent_prior']
week_prior = config['week_prior']
one_day_period = config['one_day_period']
days_of_week = config['days_of_week']
pre_len = config['pre_len']
seq_len = recent_prior + week_prior

training_epoch = config['training_epoch']


True


In [6]:
batch_size = config['batch_size']
learning_rate = config['learning_rate']

#模型结构
num_of_gru_layers = config['num_of_gru_layers']
gru_hidden_size = config['gru_hidden_size']
gcn_num_filter = config['gcn_num_filter']

loaders = []
scaler = ""
train_data_shape = ""
graph_feature_shape = ""
   ################## 获取测试数据#########################
for idx, (x, y, target_times, high_x, high_y, high_target_times, scaler) in enumerate(normal_and_generate_dataset_time(
        all_data_filename,
        train_rate=train_rate,
        valid_rate=valid_rate,
        recent_prior=recent_prior,
        week_prior=week_prior,
        one_day_period=one_day_period,
        days_of_week=days_of_week,
        pre_len=pre_len)):
    if False:#前100个，没写默认是false
        x = x[:100]
        y = y[:100]
        target_times = target_times[:100]
        high_x = high_x[:100]
        high_y = high_y[:100]
        high_target_times = high_target_times[:100]

    if 'nyc' in all_data_filename:
        graph_x = x[:, :, [0, 46, 47], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))#4584*7*3*400,注意reshape和交换维度的不同
        high_graph_x = high_x[:, :, [0, 46, 47], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))#1337*7*3*400
        graph_x = np.dot(graph_x, grid_node_map)#4584*7*3*243
        high_graph_x = np.dot(high_graph_x, grid_node_map)
    if 'chicago' in all_data_filename:
        graph_x = x[:, :, [0, 39, 40], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))
        high_graph_x = high_x[:, :, [0, 39, 40], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))
        graph_x = np.dot(graph_x, grid_node_map)
        high_graph_x = np.dot(high_graph_x, grid_node_map)

    print("feature:", str(x.shape), "label:", str(y.shape), "time:", str(target_times.shape),
            "high feature:", str(high_x.shape), "high label:", str(high_y.shape))
    print("graph_x:", str(graph_x.shape),
            "high_graph_x:", str(high_graph_x.shape))
    if idx==2:
        np.save('tset_festure.npy',x)
    if x.shape[0] ==1080:
        b = np.ones_like(x)
        b = np.where(x>1,1,0)
        print('x的最大值:{}'.format(np.sum(b)))
        print('x的最小值:{}'.format(np.min(x)))
    if idx == 0:
        scaler = scaler
        train_data_shape = x.shape
        time_shape = target_times.shape
        graph_feature_shape = graph_x.shape
    loaders.append(Data.DataLoader(
        Data.TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(target_times),#4584*32
            torch.from_numpy(graph_x),
            torch.from_numpy(y)
        ),
        batch_size=batch_size,
        shuffle=(idx == 0)
    ))
    if idx == 2:
        high_test_loader = Data.DataLoader(
            Data.TensorDataset(
                torch.from_numpy(high_x),
                torch.from_numpy(high_target_times),
                torch.from_numpy(high_graph_x),
                torch.from_numpy(high_y)
            ),
            batch_size=batch_size,
            shuffle=(idx == 0)
        )
train_loader, val_loader, test_loader = loaders
################ 获取测试数据##################
nums_of_filter = []
for _ in range(2):
    nums_of_filter.append(gcn_num_filter)

GSNet_Model = GSNet(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
                    gru_hidden_size, time_shape[1], graph_feature_shape[2],
                    nums_of_filter, north_south_map, west_east_map)
print("模型参数---------------------------------------------")
print(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
        gru_hidden_size, time_shape[1], graph_feature_shape[2],
        nums_of_filter)


# multi gpu
if torch.cuda.device_count() > 10:
    print("Let's use", torch.cuda.device_count(), "GPUs!", flush=True)
    GSNet_Model = nn.DataParallel(GSNet_Model)
############### 载入模型######################
GSNet_Model.to(device)
# print(GSNet_Model)


feature: (4584, 7, 48, 20, 20) label: (4584, 1, 20, 20) time: (4584, 32) high feature: (1337, 7, 48, 20, 20) high label: (1337, 1, 20, 20)
graph_x: (4584, 7, 3, 243) high_graph_x: (1337, 7, 3, 243)
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:7
x的最小值:0.0
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:3
x的最小值:0.0
模型参数---------------------------------------------
48 5 7 1 256 32 3 [64, 64]


GSNet(
  (st_geo_module): STGeoModule(
    (grid_conv): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(64, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
    )
    (grid_gru): GRU(48, 256, num_layers=5, batch_first=True)
    (grid_att_fc1): Linear(in_features=256, out_features=1, bias=True)
    (grid_att_fc2): Linear(in_features=32, out_features=7, bias=True)
    (grid_att_softmax): Softmax(dim=-1)
  )
  (st_sem_module): STSemModule(
    (road_gcn): ModuleList(
      (0): GCN_Layer(
        (gcn_layer): Sequential(
          (0): Linear(in_features=3, out_features=64, bias=True)
          (1): ReLU()
        )
      )
      (1): GCN_Layer(
        (gcn_layer): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
      )
    )
    (risk_gcn): ModuleList(
      (0): GCN_Layer(
        (gcn_layer): Sequential(
          (0): Linea

In [7]:

#GSNet_Model.load_state_dict(torch.load("data/model_parameter.pkl"))

#GSNet_Model.load_state_dict(torch.load("/home/wyb/mycode/GSNet-master/data/last_model03.pt"))
#GSNet_Model.load_state_dict(torch.load("data/last_model.pt"))
GSNet_Model.load_state_dict(torch.load("data/model38.pt"))
print("模型加载成功！")

num_of_parameters = 0
for name, parameters in GSNet_Model.named_parameters():
    num_of_parameters += np.prod(parameters.shape)
print("Number of Parameters: {}".format(num_of_parameters), flush=True)

trainer = optim.Adam(GSNet_Model.parameters(), lr=learning_rate)
early_stop = EarlyStopping(patience=patience, delta=delta)

risk_mask = get_mask(mask_filename)
road_adj = get_adjacent(road_adj_filename)
risk_adj = get_adjacent(risk_adj_filename)
if poi_adj_filename == "":
    poi_adj = None
else:
    poi_adj = get_adjacent(poi_adj_filename)


模型加载成功！
Number of Parameters: 6278578


In [8]:
#配置log日志输出
from myfunction import get_root_logger, logger_info
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")


2023-05-08 17:09:36,434 - INFO - Loading config file from 


In [9]:
torch.cuda.empty_cache()

In [10]:
#攻击，并写入数据,20230315导入包修改为lib.utils_new_round
import yaml
from mmcv import Config
from myfunction import log_test_results
from lib.Imperceptible_attack import log_test_csv,node_map,prenoise_pgd,rcr_pgd,fgsm,min,kl_pgd,JS_pgd_impr,rand_attack,nonenoise_pgd
from lib.Imperceptible_attack import ZINBSC,fgsm_impr,min_impr,pgd_impr,random_impr,log_test_csv_time_K,ZINBSC_geneater_kk
from myfunction import get_root_logger, logger_info,kl_div_pro
from lib.admm_atack_new import none_admm_attack
from lib.distru_attack import none_distri_attack
from lib.vers_attack import vers_attack
from lib.select import select_attack
#配置logo
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")
cfgin = Config.fromfile('attack_setting.yaml')
#根据配置选择攻击方法
if cfgin.attacker == 'white_attacker':
    header = ['time','epoch','dataset', 'model','node_select', 'method', 'K','batch size', 
              'clean_RMSE', 'clean_recall', 'clean_MAP', 'clean_RCR',
              'adv_RMSE', 'adv_recall', 'adv_MAP', 'adv_RCR',
              'local_adv_RMSE', 'local_adv_recall', 'local_adv_MAP', 'local_adv_RCR',
              '备注信息：特征分析，50']
    #文件名
    file_name = '第28次-{}-eps{}-model-{}'.format(
        cfgin.dataset, cfgin.test_epsilon, cfgin.backbone)
    log_test_results(cfgin.result_dir, header, file_name)
    #可能的时间循环
    #攻击方法+节点选择组合
    for epoch in range(3):
        #break
        for out_K in cfgin.K:
            for i in cfgin.select_node:
                for j in cfgin.attack_method:
                    for chi_k in range(35,36):
                        logger.info(i+'_'+j+"_"+str(out_K)+"_"+str(chi_k))
                        parm_att = j #攻击方法的函数名称
                        parm_node = i #节点选择的方法
                        #进行攻击,输出攻击日志
                        # 1. 节点选择，返回攻击的节点
                        ack_map = node_map(i,chi_k,out_K, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                            grid_node_map, device, data_type='nyc')
                        # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
                        adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                                grid_node_map, scaler, risk_mask, device, data_type='nyc')
                        #计算结果并打印、存储,注意表头信息需要传递参数
                        log_test_csv_time_K(epoch,out_K,adv_val_predict, val_target,
                                            val_predict,cfgin,i,j,file_name,risk_mask)
                        torch.cuda.empty_cache()
    for epoch in range(1):
        for out_K in [20,30,40]:
            i = 'ten'
            j = 'ZINBSC'
            for chi_k in range(26,45):
                logger.info(i+'_'+j+"_"+str(out_K)+"_"+str(chi_k))
                parm_att = j #攻击方法的函数名称
                parm_node = i #节点选择的方法
                #进行攻击,输出攻击日志
                # 1. 节点选择，返回攻击的节点
                ack_map = node_map(i,chi_k,out_K, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                    grid_node_map, device, data_type='nyc')
                # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
                adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                        grid_node_map, scaler, risk_mask, device, data_type='nyc')
                #计算结果并打印、存储,注意表头信息需要传递参数
                log_test_csv_time_K(epoch,out_K,adv_val_predict, val_target,
                                    val_predict,cfgin,i,j,file_name,risk_mask)
                torch.cuda.empty_cache()
           

2023-05-08 17:09:36,603 - INFO - Loading config file from 
2023-05-08 17:09:36,607 - INFO - ten_ZINBSC_40_35


time,epoch,dataset,model,node_select,method,K,batch size,clean_RMSE,clean_recall,clean_MAP,clean_RCR,adv_RMSE,adv_recall,adv_MAP,adv_RCR,local_adv_RMSE,local_adv_recall,local_adv_MAP,local_adv_RCR,备注信息：特征分析，50


2023-05-08 17:10:17,050 - INFO - Info:  [64/1080 (6%)]	    adv_RMSE: 0.9317 adv_Recall: 28.1787 adv_MAP: 0.1252 adv_RCR: 0.7444 time:4.955
2023-05-08 17:10:27,224 - INFO - Info:  [128/1080 (12%)]	    adv_RMSE: 0.8686 adv_Recall: 28.4404 adv_MAP: 0.1467 adv_RCR: 0.7396 time:4.941
2023-05-08 17:10:37,411 - INFO - Info:  [192/1080 (18%)]	    adv_RMSE: 0.8663 adv_Recall: 25.7812 adv_MAP: 0.1341 adv_RCR: 0.7459 time:4.904
2023-05-08 17:10:47,571 - INFO - Info:  [256/1080 (24%)]	    adv_RMSE: 0.9624 adv_Recall: 32.6087 adv_MAP: 0.1466 adv_RCR: 0.7119 time:4.946
2023-05-08 17:10:57,713 - INFO - Info:  [320/1080 (29%)]	    adv_RMSE: 0.8519 adv_Recall: 20.6349 adv_MAP: 0.0953 adv_RCR: 0.8097 time:4.955
2023-05-08 17:11:07,936 - INFO - Info:  [384/1080 (35%)]	    adv_RMSE: 0.8791 adv_Recall: 29.0179 adv_MAP: 0.1230 adv_RCR: 0.7526 time:4.961
2023-05-08 17:11:18,134 - INFO - Info:  [448/1080 (41%)]	    adv_RMSE: 0.9140 adv_Recall: 25.9542 adv_MAP: 0.1265 adv_RCR: 0.7345 time:4.961
2023-05-08 17:1

2023-05-08 17:13:23,0,NYC,Gsnet,ten,ZINBSC,40,32,0.6847454450474136,32.43045387994143,0.18557500325210685,0.6796269542597637,0.8771139498263947,26.976573938506586,0.12234510092904026,0.7448081802001326,0.5646083272114971,99.69123172501023,0.9934309128082572,0.0013230829610785387


2023-05-08 17:14:10,740 - INFO - Info:  [64/1080 (6%)]	    adv_RMSE: 0.9370 adv_Recall: 26.8041 adv_MAP: 0.1247 adv_RCR: 0.7792 time:6.095
2023-05-08 17:14:23,230 - INFO - Info:  [128/1080 (12%)]	    adv_RMSE: 0.8697 adv_Recall: 28.8991 adv_MAP: 0.1105 adv_RCR: 0.7480 time:6.120
2023-05-08 17:14:35,598 - INFO - Info:  [192/1080 (18%)]	    adv_RMSE: 0.8675 adv_Recall: 26.5625 adv_MAP: 0.1497 adv_RCR: 0.7479 time:6.020
2023-05-08 17:14:47,991 - INFO - Info:  [256/1080 (24%)]	    adv_RMSE: 0.9613 adv_Recall: 31.5217 adv_MAP: 0.1471 adv_RCR: 0.7217 time:6.029
2023-05-08 17:15:00,462 - INFO - Info:  [320/1080 (29%)]	    adv_RMSE: 0.8450 adv_Recall: 20.6349 adv_MAP: 0.0990 adv_RCR: 0.8101 time:6.119
2023-05-08 17:15:12,876 - INFO - Info:  [384/1080 (35%)]	    adv_RMSE: 0.8782 adv_Recall: 27.2321 adv_MAP: 0.1174 adv_RCR: 0.7700 time:6.089
2023-05-08 17:15:25,271 - INFO - Info:  [448/1080 (41%)]	    adv_RMSE: 0.9150 adv_Recall: 26.7176 adv_MAP: 0.1224 adv_RCR: 0.7130 time:6.045
2023-05-08 17:1

...

2023-05-08 20:30:37,0,NYC,Gsnet,ten,ZINBSC,40,32,0.6847454450474136,32.43045387994143,0.18557500325210685,0.6796269542597637,0.8611372987097479,27.318204001952175,0.12629451741097839,0.7412477712143604,0.5464631590680785,99.68565157546222,0.9934030306190615,0.0016247741261450708


2023-05-08 20:31:19,360 - INFO - Info:  [64/1080 (6%)]	    adv_RMSE: 0.9192 adv_Recall: 28.1787 adv_MAP: 0.1211 adv_RCR: 0.7623 time:4.789
2023-05-08 20:31:29,138 - INFO - Info:  [128/1080 (12%)]	    adv_RMSE: 0.8686 adv_Recall: 30.7339 adv_MAP: 0.1742 adv_RCR: 0.6922 time:4.807
2023-05-08 20:31:39,094 - INFO - Info:  [192/1080 (18%)]	    adv_RMSE: 0.8847 adv_Recall: 25.3906 adv_MAP: 0.1243 adv_RCR: 0.7607 time:4.884
2023-05-08 20:31:49,002 - INFO - Info:  [256/1080 (24%)]	    adv_RMSE: 0.9542 adv_Recall: 33.6957 adv_MAP: 0.1704 adv_RCR: 0.6996 time:4.893
2023-05-08 20:31:58,990 - INFO - Info:  [320/1080 (29%)]	    adv_RMSE: 0.8499 adv_Recall: 19.5767 adv_MAP: 0.0990 adv_RCR: 0.8249 time:4.956
2023-05-08 20:32:08,925 - INFO - Info:  [384/1080 (35%)]	    adv_RMSE: 0.8628 adv_Recall: 26.7857 adv_MAP: 0.1281 adv_RCR: 0.7627 time:4.881
2023-05-08 20:32:18,887 - INFO - Info:  [448/1080 (41%)]	    adv_RMSE: 0.9264 adv_Recall: 27.0992 adv_MAP: 0.1345 adv_RCR: 0.7371 time:4.925
2023-05-08 20:3

2023-05-08 20:33:59,0,NYC,Gsnet,ten,ZINBSC,40,32,0.6847454450474136,32.43045387994143,0.18557500325210685,0.6796269542597637,0.8764330723524666,26.98877501220107,0.12634511263920953,0.7467021461862293,0.5760345542371837,99.69123172501023,0.9934341187672567,0.0019098973711799872


2023-05-08 20:34:41,551 - INFO - Info:  [64/1080 (6%)]	    adv_RMSE: 0.9063 adv_Recall: 28.1787 adv_MAP: 0.1198 adv_RCR: 0.7628 time:4.890
2023-05-08 20:34:51,515 - INFO - Info:  [128/1080 (12%)]	    adv_RMSE: 0.8591 adv_Recall: 31.1927 adv_MAP: 0.1655 adv_RCR: 0.7025 time:4.894
2023-05-08 20:35:01,445 - INFO - Info:  [192/1080 (18%)]	    adv_RMSE: 0.8711 adv_Recall: 25.0000 adv_MAP: 0.1378 adv_RCR: 0.7252 time:4.869
2023-05-08 20:35:11,288 - INFO - Info:  [256/1080 (24%)]	    adv_RMSE: 0.9383 adv_Recall: 33.4239 adv_MAP: 0.1581 adv_RCR: 0.7118 time:4.853
2023-05-08 20:35:21,160 - INFO - Info:  [320/1080 (29%)]	    adv_RMSE: 0.8334 adv_Recall: 23.8095 adv_MAP: 0.1134 adv_RCR: 0.7594 time:4.869
2023-05-08 20:35:31,056 - INFO - Info:  [384/1080 (35%)]	    adv_RMSE: 0.8566 adv_Recall: 27.2321 adv_MAP: 0.1560 adv_RCR: 0.7170 time:4.866
2023-05-08 20:35:40,949 - INFO - Info:  [448/1080 (41%)]	    adv_RMSE: 0.9097 adv_Recall: 25.5725 adv_MAP: 0.1341 adv_RCR: 0.7269 time:4.890
2023-05-08 20:3

2023-05-08 20:37:31,0,NYC,Gsnet,ten,ZINBSC,40,32,0.6847454450474136,32.43045387994143,0.18557500325210685,0.6796269542597637,0.8629145734873402,27.2938018545632,0.12771663779772197,0.7399409463051567,0.552780646829183,99.67635132621554,0.9934069489807621,0.001989377590228727


In [11]:
import pandas as pd
aso = np.load('riak_243.npy')
bso = np.load('map_0502.npy')

print(aso)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [12]:
print(aso[38,:,:])
print(np.sum(aso[38,:,:]))

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:

print(bso[100,1,0,:,:]*46.0)

In [ ]:
map_243 = np.load("data/nyc/243nyc.npy")
print(map_243.shape)



In [ ]:
import torch
map_a = torch.randint(0,2,(20,20))
map_b = torch.randint(10,11,(1,10))

idx = torch.nonzero(map_a  == 1)
if idx.shape[0]>20:
    map_a[idx[:10,:]]==map_b 
print(map_a)



In [ ]:
import torch
a=torch.randint(1,3,(2,3))
b=torch.randint(1,3,(2,3))
c = []
c.append(a)
c.append(b)

print(torch.stack(c).shape)

In [ ]:
import  numpy as np
a=np.random.randint(1,3,(2,3))
b=np.random.randint(1,3,(2,3))
c = []
c.append(a)
c.append(b)

print(np.vstack(c))
print(np.concatenate([a,b],axis=0))
print(np.concatenate(c,axis=0))